In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install geopy

In [ ]:
pip install wget

In [25]:
import urllib.request, urllib.parse, urllib.error
import ssl
import numpy as np 
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from bs4 import BeautifulSoup
import wget

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# we use Beautiful soup for parsing the wiki page
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# look for the table we need, with class wikitable sortable
table_w = soup.find('table','wikitable sortable')

In [5]:
# look for the cell headers with <th> element
th_1 = table_w.tr.find_all("th")
titulos = list()
for element in th_1:
    element = str(element)
    element = element.replace('<th>','').replace('</th>','').replace('\n','')
    titulos.append(element)
titulos

['Postal Code', 'Borough', 'Neighborhood']

In [6]:
# look for the table data with <tr> 
th_2 = table_w.find_all("tr")
# we delete the first list that contains the headers
th_2 = th_2[1:]
# convert the html code to text str, we edit the text we dont need "<tr>, </tr> and \n". We split each row with <td> 
# select the only elements in the list we are interested
data = list()
for lista in th_2:
    lista = str(lista)
    lista = lista.replace('<tr>','').replace('</tr>','').replace('</td>','').replace('\n','')
    lista = lista.split('<td>')
    lista = lista[1:4]
    data.append(lista)

In [7]:
# we convert list to a dataframe
data_1 = pd.DataFrame(data)
# add the titles and delete a 4th column generated by error
data_1.columns = titulos
# we drop Borough which are not assigned
data_1.set_index('Borough', inplace=True) 
data_1.drop(["Not assigned"], inplace = True) 
data_1.reset_index(inplace=True)

In [8]:
# control if we have a 'Not assigned' Neighborhood
data_1_no = data_1[data_1['Neighborhood']=='Not assigned']
print(data_1_no.shape)

(0, 3)


In [9]:
# check 
data_1.shape

(103, 3)

In [30]:
# check the look
data_1.set_index('Postal Code', inplace=True) 
data_1

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


In [11]:
# TEST - DELETE
# create a list of postal codes
postal_code = data_1['Postal Code'].to_list()

In [20]:
# TEST - DELETE
'Toronto, Ontario, '+postal_code[0]+', Canada'

'Toronto, Ontario, M3A, Canada'

In [21]:
# TEST - DELETE
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode('Toronto, Ontario,'+postal_code[0]+' Canada')

In [22]:
# TEST - DELETE
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 43.6534817, Longitude = -79.3839347


In [23]:
# TEST - DELETE
location

Location(Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada, (43.6534817, -79.3839347, 0.0))

In [37]:
url = 'https://cocl.us/Geospatial_data'
filename = wget.download(url)   

100% [................................................................................] 2891 / 2891

In [38]:
location_2 = pd.read_csv(filename, delimiter=",")

In [39]:
location_2.set_index('Postal Code', inplace=True)

In [40]:
location_2

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [41]:
data_1.join(location_2)

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
